In [1]:
import requests
import json
import pandas as pd
import psycopg2

In [2]:
def get_data():
    """
    get data from random user api
    """
    try:
        url = "https://randomuser.me/api/"
        response = requests.get(url)
        response = json.loads(response.text)
        return response["results"][0]
    except Exception as e:
        print(f"there was an error{e}")
        return None

In [3]:
people_list = []

for _ in range(5):
    people_list.append(get_data())

In [4]:
data = pd.DataFrame(people_list,index=None)

In [5]:
data.head(5)

,gender,name,location,email,login,dob,registered,phone,cell,id,picture,nat
0,female,"{'title': 'Ms', 'first': 'Alisa', 'last': 'Kivi'}","{'street': {'number': 6837, 'name': 'Esplanadi...",alisa.kivi@example.com,{'uuid': '05eca122-4dae-4a39-ada5-93fa6c1b3d79...,"{'date': '1997-12-30T15:31:43.645Z', 'age': 26}","{'date': '2015-04-04T02:55:48.803Z', 'age': 9}",08-071-846,049-912-21-19,"{'name': 'HETU', 'value': 'NaNNA246undefined'}",{'large': 'https://randomuser.me/api/portraits...,FI
1,female,"{'title': 'Miss', 'first': 'Stefanija', 'last'...","{'street': {'number': 8802, 'name': 'Udarnih B...",stefanija.tadic@example.com,{'uuid': 'cc34ee6a-a5b9-4ccf-8e5e-bd5e93a6d27a...,"{'date': '1964-12-07T19:28:00.931Z', 'age': 59}","{'date': '2020-11-18T02:36:54.481Z', 'age': 3}",016-4814-643,062-4531-587,"{'name': 'SID', 'value': '748800857'}",{'large': 'https://randomuser.me/api/portraits...,RS
2,female,"{'title': 'Miss', 'first': 'Kristin', 'last': ...","{'street': {'number': 9707, 'name': 'Mockingbi...",kristin.pena@example.com,{'uuid': '8b04f833-f1d8-482a-940d-2e018f46067c...,"{'date': '1946-12-03T18:19:19.422Z', 'age': 77}","{'date': '2003-07-07T02:32:05.012Z', 'age': 20}",09-2404-4726,0469-501-338,"{'name': 'TFN', 'value': '197972747'}",{'large': 'https://randomuser.me/api/portraits...,AU
3,female,"{'title': 'Mrs', 'first': 'Tverdogosta', 'last...","{'street': {'number': 6722, 'name': 'Patona'},...",tverdogosta.radkevich@example.com,{'uuid': 'eb7faa93-be1b-4530-921c-9233dbaa9792...,"{'date': '1956-06-19T04:34:13.379Z', 'age': 67}","{'date': '2006-08-30T03:03:49.878Z', 'age': 17}",(097) Q65-4515,(067) J45-7597,"{'name': '', 'value': None}",{'large': 'https://randomuser.me/api/portraits...,UA
4,female,"{'title': 'Ms', 'first': 'Camille', 'last': 'J...","{'street': {'number': 3117, 'name': '20th Ave'...",camille.jones@example.com,{'uuid': '47d140b9-6663-4b43-bfd0-157f5cc855e0...,"{'date': '1969-11-04T09:35:13.524Z', 'age': 54}","{'date': '2021-07-25T22:33:15.704Z', 'age': 2}",V75 I39-0648,C42 I80-9695,"{'name': 'SIN', 'value': '155009103'}",{'large': 'https://randomuser.me/api/portraits...,CA


In [6]:
query = """CREATE TABLE test_table (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    email TEXT UNIQUE
)"""

In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourAppName") \
    .config("spark.driver.extraClassPath", "postgresql-connector-java.jar") \
    .config("spark.executor.extraClassPath", "postgresql-connector-java.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 21:05:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
# url
url = "http://postgres-db:45432/postgres_db"
#user
username = "postgres"
password = "postgres"

In [21]:
conn_details = psycopg2.connect(
    host='postgres-db',
    database="postgres_db",
    user=username,
    # secretlint-disable
    password=password,
    # secretlint-enable
    port='5432'
)

with conn_details:
    cursor = conn_details.cursor()
    query = query
    cursor.execute(query)

In [25]:
spark_df = spark.createDataFrame(data)

In [26]:
spark_df.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+--------------------+--------------------+---+
|gender|                name|            location|               email|               login|                 dob|          registered|         phone|          cell|                  id|             picture|nat|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+--------------------+--------------------+---+
|female|{last -> Kivi, ti...|{country -> NULL,...|alisa.kivi@exampl...|{sha1 -> 5352e4c1...|{date -> 1997-12-...|{date -> 2015-04-...|    08-071-846| 049-912-21-19|{name -> HETU, va...|{thumbnail -> htt...| FI|
|female|{last -> Tadić, t...|{country -> NULL,...|stefanija.tadic@e...|{sha1 -> f304fbd7...|{date -> 1964-12-...|{date -> 2020-11-...|  016-4814-643|  062-4

In [29]:
properties = {"user": "postgres","password": password}
spark_df.write.jdbc(url='postgres-db', table="test_result", properties=properties)

Py4JJavaError: An error occurred while calling o55.jdbc.
: java.lang.ClassNotFoundException: postgresql-connector-java.jar
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:258)

In [3]:
import mysql.connector

def create_mysql_connection():
    try:
        # Connect to the MySQL database
        connection = mysql.connector.connect(
            host="localhost",
            user="username",
            password="password",
            database="database_name"
        )
        print("Connection to MySQL database successful.")
        return connection
    except mysql.connector.Error as e:
        print("Error connecting to MySQL database:", e)
        return None

if __name__ == "__main__":
    connection = create_mysql_connection()
    if connection:
        connection.close()
        print("Connection closed.")


Error connecting to MySQL database: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (99)


In [30]:
! ls

ETL.ipynb  docker-compose.yml


In [31]:
! cd ../

In [32]:
! ls

ETL.ipynb  docker-compose.yml
